In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
import ipywidgets as ipw
from IPython.display import clear_output

# AiiDA imports.
%aiida

# AiiDAlab imports.
import aiidalab_widgets_base as awb

# Local imports.
from reactions.dft_details_dev import DFTDetails
from reactions.viewer_details import ViewerDetails
from reactions.slab_validity import slab_is_valid
from reactions.suggested_param import suggested_parameters
from reactions import analyze_structure
from reactions.metadata import MetadataWidget
from reactions.replicawork import ReplicaWorkChain
from reactions.collective_variables import CollectiveVariableWidget

In [ ]:
# General parameters.

job_details = {'workchain':'ReplicaWorkChain'}
atoms = None

In [ ]:
## WIDGETS MONITOR FUNCTIONS
def on_struct_change(c):
    global atoms
    if not struct_browser.results.value:
        return
    job_details['structure']=struct_browser.results.value
    node = struct_browser.results.value
    
    atoms = node.get_ase()
    atoms.pbc = [1, 1, 1]
    
    slab_analyzed = analyze_structure.analyze(atoms)
    job_details['slab_analyzed'] = slab_analyzed
    dft_details_widget.reset()
    
    guess_calc_params(slab_analyzed)
    
    viewer_widget.setup(atoms, slab_analyzed)
    
    with mol_ids_info_out:
        clear_output()
        print(slab_analyzed['summary'])

def on_fixed_atoms_btn_click(c):
    if dft_details_widget.btn_fixed_pressed:
        viewer_widget.show_fixed(dft_details_widget.fixed_atoms.value)
    else:
        viewer_widget.show_fixed("")
        
def guess_calc_params(slab_analyzed):
    method = dft_details_widget.calc_type.value
    valid_slab, msg = slab_is_valid(slab_analyzed,method)
    if valid_slab:        
        atoms_to_fix,num_machines=suggested_parameters(slab_analyzed,method)
        dft_details_widget.reset(fixed_atoms=atoms_to_fix,calc_type=method)
        num_nodes_selector.value = num_machines
    else:
        print(msg)

## Step 1: Select initial structure

In [ ]:
# DISPLAY WIDGETS AND DEFINE JOB PARAMETERS

# Structure.
struct_browser = awb.StructureBrowserWidget()
struct_browser.results.observe(on_struct_change, names='value')

# Viewer.
viewer_widget = ViewerDetails()

# Mol info.
mol_ids_info_out = ipw.Output()

display(ipw.VBox([struct_browser, viewer_widget, mol_ids_info_out]))

## Step 2: General parameters of the calculation

In [ ]:
# Code.
computational_resources = awb.ComputationalResourcesWidget(input_plugin='cp2k')

# Misc details.
style = {'description_width': '120px'}
layout = {'width': '70%'}
num_nodes_selector = ipw.IntText(
    value=1,
    description='# Nodes',
    style=style, layout=layout)
calc_name_selector = ipw.Text(
    description='Calculation Name: ',
    placeholder='A great name.',
    style=style, layout=layout)

# DFT
dft_details_widget = DFTDetails(job_details=job_details)    
dft_details_widget.btn_fixed_atoms.on_click(on_fixed_atoms_btn_click)
dft_details_widget.calc_type.observe(lambda c: guess_calc_params(), names='value')

# Display code dropdown.
display(ipw.VBox([computational_resources, num_nodes_selector, calc_name_selector, dft_details_widget]))

## Step 3: Define collective variable
The unit of the spring constant is [eV/unit of colvar^2].

Three kinds of collective variable are implemented:

**DISTANCE**
- Distance between two atoms.

**ANGLE_PLANE_PLANE**
- Rotation of plane defined by (P1 - P2) and (P3 - P2) WRT plane defined by a normal.

**BOND_ROTATION**
- Clockwise rotation of (P2-P1) WRT (P4-P3).

In [ ]:
cv_widget = CollectiveVariableWidget()

def on_vis_cv_click(c):
    if cv_widget.validation_check() and atoms is not None:
        vis_list = cv_widget.current_cv_instance.visualization_list(atoms)
        viewer_widget.visualize_extra(vis_list)
        
        with mol_ids_info_out:
            cv_widget.current_cv_instance.print_cv(atoms)
        
cv_widget.visualize_colvar_btn.on_click(on_vis_cv_click)

display(cv_widget)

Optionally, to continue another replica chain calculation, use the exact same calculation name, pick one of its output geometries as input and use the same collective variable definition.

# Submit

In [ ]:
def build_inputs():
    builder = ReplicaWorkChain.get_builder()
    builder.cp2k_code = computational_resources.value
    builder.structure = struct_browser.results.value
    builder.num_machines = Int(num_nodes_selector.value)
    builder.calc_name = Str(calc_name_selector.value)
    #builder.cell
    builder.fixed_atoms = Str(dft_details_widget.fixed_atoms.value)
    builder.colvar_targets = Str(cv_widget.colvar_targets)
    builder.target_unit = Str(cv_widget.target_unit)
    builder.spring = Float(cv_widget.spring_constant)
    builder.spring_unit = Str(cv_widget.spring_unit)
    builder.subsys_colvar = Dict(dict=cv_widget.subsys_colvar)
    builder.calc_type = Str(dft_details_widget.calc_type.value)
    builder.mgrid_cutoff = Int(dft_details_widget.mgrid_cutoff.value)
    builder.max_force = Float(dft_details_widget.max_force.value)
    builder.dftd3_switch = Bool(dft_details_widget.vdw_switch.value)

    builder.metadata = {
        "description": calc_name_selector.value,
        "label": "ReplicaWorkChain",
    }
    return builder

In [ ]:
btn_submit = awb.SubmitButtonWidget(ReplicaWorkChain,input_dictionary_function=build_inputs)
submit_out = ipw.Output()
display(btn_submit, submit_out)